In [ ]:
# https://huggingface.co/blog/lcm_lora
# https://huggingface.co/segmind/SSD-1B
import datetime
import os
import time
import warnings

# On Intel without a GPU, the cuda backend will spew errors and warnings.
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")

import diffusers
import torch
from IPython.display import display

DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE in ("cuda", "mps") else torch.float32
print(f"Using {DEVICE}")

def get_generator(seed):
    if DEVICE in ("cuda", "mps"):
        return torch.Generator(DEVICE).manual_seed(seed)
    generator = torch.Generator()
    generator.manual_seed(seed)
    return generator

pipe = diffusers.DiffusionPipeline.from_pretrained("segmind/SSD-1B")
pipe.load_lora_weights("latent-consistency/lcm-lora-ssd-1b")
pipe.scheduler = diffusers.LCMScheduler.from_config(pipe.scheduler.config)
pipe.to(DEVICE, dtype=DTYPE)

if DEVICE == "cuda":
    torch.backends.cuda.matmul.allow_tf32 = True
elif DEVICE == "cpu":
    try:
        # Interestingly I see no performance improvement at all.
        import intel_extension_for_pytorch as ipex
        pipe.unet = ipex.optimize(pipe.unet.eval(), dtype=DTYPE, inplace=True)
        pipe.vae = ipex.optimize(pipe.vae.eval(), dtype=DTYPE, inplace=True)
        pipe.text_encoder = ipex.optimize(pipe.text_encoder.eval(), dtype=DTYPE, inplace=True)
        print("Using Intel (accelerated)")
    except ImportError:
        pass

def gen_image(prompt, seed):
    return pipe(
        prompt=prompt, num_inference_steps=4,
        guidance_scale=1, generator=get_generator(seed)).images[0]

In [ ]:
#prompt = "close-up photography of old man standing in the rain at night, in a street lit by lamps, leica 35mm summilux"
#prompt = "scene from the 1979 dystopian action film Mad Max, 35mm Eastmancolor"
prompt = "fireplace, warm cozy book shelf candle snow mountain window sofa rustic"
seed = 30

start = time.time()
for i in range(5):
    img = gen_image(prompt, seed+i)
    print(f"{i} Inference in {time.time()-start:.1f}s")
    display(img)
    #now = str(datetime.datetime.now().replace(microsecond=0)).replace(" ", "-").replace(":", "-")
    #img.save(f"out/ssd1b_lcm_lora-{now}.png")